In [2]:
import pandas as pandas
import nltk
import time
import re
import numpy as np

import warnings
warnings.filterwarnings(action='ignore', category=UserWarning, module='gensim')

In [3]:
filename = 'train_data.csv'
t1 = time.time()
data = pandas.read_csv('train_data.csv')

In [4]:
nreviews=10000
#nreviews=data.shape[0]

In [5]:
data_txt=data["text"]
data_txt=data_txt[0:nreviews]

In [6]:
from nltk.corpus import stopwords

In [7]:
stop = set(stopwords.words("english"))
stop_neg=["wasn't",'wasn',"couldn't",'couldn',
          "wouldn't",'wouldn',"shouldn't",'shouldn',"shan't",
          "haven't","hasn't","hadn't","hadn'",
          "don't",'don',"doesn't",'doesn',"didn't",
          "needn't",'needn','no','nor',
          "isn't",'isn',"aren't","weren't",'weren',"won't"]
stop_revised=[w for w in stop if not w in stop_neg]

In [8]:
def stop_delete(sentence):
    sentence=sentence.lower().split(" ")
    sen=[word for word in sentence if not word in stop]
    sen_re=np.array([])
    for j in sen:
        word_re=re.sub('[^a-zA-Z]','', j)
        if word_re != "":
            sen_re=np.append(sen_re,word_re)
    return sen_re

def stop_delete_re(sentence):
    sentence=sentence.lower().split(" ")
    sen=[word for word in sentence if not word in stop_revised]
    sen_re=np.array([])
    for j in sen:
        if j in stop_neg:
            sen_re=np.append(sen_re,"neg_w")
        else:
            word_re=re.sub('[^a-zA-Z]','', j)
            if word_re != "":
                sen_re=np.append(sen_re,word_re)
    return sen_re

In [9]:
from collections import defaultdict
from nltk.stem.wordnet import WordNetLemmatizer 
from nltk.stem.porter import PorterStemmer 

In [10]:
lem = WordNetLemmatizer()
stem = PorterStemmer()

#clear text
document=[]
document_re=[]
for sen in data_txt:
    txt=stop_delete(sen)
    txt_re=stop_delete_re(sen)
    
    txt=[stem.stem(word) for word in txt]
    txt_re=[stem.stem(word) for word in txt_re]
    
    document.append(txt)
    document_re.append(txt_re)

In [11]:
#frequency, threshold=min_freq
frequency=defaultdict(int)
#intensity
freq_pos=defaultdict(int)
freq_neg=defaultdict(int)

score=data["stars"]

for i in range(nreviews):
    sen=document[i]
    for word in sen:
        frequency[word] += 1
        if score[i] > 3:
            freq_pos[word] += 1
        elif score[i] < 3:
            freq_neg[word] += 1

In [12]:
min_freq=100

In [13]:
document=[[word for word in sen if frequency[word] > min_freq] for sen in document]
document_re=[[word for word in sen if frequency[word] > min_freq] for sen in document_re]

In [14]:
frequency_dict={}
for word in list(frequency):
    if word not in list(freq_pos):
        freq_pos[word]=0
    if word not in list(freq_neg):
        freq_neg[word]=0    
    if frequency[word] <= min_freq:
        del frequency[word]
        del freq_pos[word]
        del freq_neg[word]
    else:
        frequency_dict[word]=[frequency[word],freq_pos[word],freq_neg[word]]

In [35]:
data_w=data[0:nreviews]
data_w=data_w.drop("text", axis=1)

text=[]

for i in range(nreviews):
    sen=document_re[i]
    txt=""
    for w in sen:
        txt=txt+w+" "
    text.append(txt)

data_w["text"]=text    
        
data_w.to_csv("data_cleaned.csv", sep=',')

In [44]:
import csv

In [57]:
with open('document.csv', 'w', newline="") as myfile:
    writer = csv.writer(myfile)
    writer.writerows(document)
    
with open('document_re.csv', 'w', newline="") as myfile:
    writer = csv.writer(myfile)
    writer.writerows(document_re)

with open('frequency.csv', 'w', newline="") as myfile:
    writer = csv.writer(myfile)
    writer.writerow(["word","count","pos","neg"])
    for key, value in frequency_dict.items():
        writer.writerow([key, value[0],value[1],value[2]])

In [60]:
with open("document.csv", 'r') as filein: 
    reader = csv.reader(filein)
    document = list(list(rec) for rec in csv.reader(filein, delimiter=',')) 
    
freq_std = pandas.read_csv('frequency.csv')

In [61]:
freq_std["pos_std"]=freq_std["pos"]/freq_std["count"]
freq_std["neg_std"]=freq_std["neg"]/freq_std["count"]
print(freq_std.head())

      word  count   pos  neg   pos_std   neg_std
0  serious    156    79   62  0.506410  0.397436
1   cannot    142    88   34  0.619718  0.239437
2    stand    200   102   67  0.510000  0.335000
3    never   1274   668  474  0.524333  0.372057
4      get   3558  1981  944  0.556773  0.265318


In [67]:
#base dict, threshold=base_n
base_num=100

base_p=freq_std.sort_values("pos_std",ascending=0)
base_pos=base_p["word"][0:base_num]

base_n=freq_std.sort_values("neg_std",ascending=0)
base_neg=base_n["word"][0:base_num]


450      fantast
460          gem
818          yum
533      perfect
816      delight
608       highli
658     outstand
879         cozi
203       delici
59          amaz
61         excel
212      favorit
79        awesom
280         glad
905      homemad
107        yummi
909        fabul
537    perfectli
914        sampl
590         rich
726     knowledg
253       wonder
314       creami
105         love
583         duck
69         great
389         beat
290         spot
639       welcom
301        juici
         ...    
501       tender
360         stuf
233        enjoy
667      varieti
597        split
675      authent
649           th
428         must
259       indian
484        thank
176      absolut
854     charlott
445     cocktail
286         town
813        vegan
653       plenti
250         vega
469     mushroom
206        spici
516      everyth
28           lol
512      environ
451         wine
846       desert
730      outdoor
391       brunch
235        roast
374       spri

        word  count  pos  neg   pos_std   neg_std
423    worst    218   14  181  0.064220  0.830275
40   terribl    260   27  205  0.103846  0.788462
753  horribl    221   23  174  0.104072  0.787330
580       aw    106   12   79  0.113208  0.745283
130     rude    198   24  147  0.121212  0.742424
